In [1]:
import os
import sys
from time import gmtime, strftime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import torch
import poutyne
from poutyne import Model,Experiment

In [3]:
from data.custom_data import filepath_dataframe
from data.selection import Selection
from data.transformation import Transform_CnnLstmS,Transform_CnnS
from data.torchData import DataLoadings,DataLoading

from data.custom_data import nucPaired_fpDataframe
from data.torchData import PairDataLoading,DataLoading
from training.contrastive_pretraining import Contrastive_PreTraining
from training.finetuning import FineTuneCNN

import models

In [4]:
#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## data directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
fpath = '.'

# data selection
dataselection_name = 'EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced'

data_selection = Selection(split='random',test_sub=0.2,val_sub=0.1,
                           nuc='NUC1',room=1,sample_per_class=None)

# data loading
transform = Transform_CnnS()
batch_size = 64
num_workers = 0

# training
optimizer_builder = torch.optim.SGD
lr = 0.001
pretrain_epochs = 1
finetune_epochs = 1

# model
builder = models.cnn.create_alexnet
network_name = 'AlexNet'

# Experiment Name
comment = 'TestLoov'
exp_name = f'{network_name}_Supervised_{dataselection_name}_Comment-{comment}'

# auto
record_dir = os.path.join(fpath,'record')
model_dir = os.path.join(fpath,'saved_model')
model_fname = os.path.join(model_dir,f'Encoder___{exp_name}')
print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())

Experiment Name:  AlexNet_Supervised_EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced_Comment-TestLoov
Cuda Availability:  True


In [5]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# data preparation
df = filepath_dataframe(data_dir,splitchar)
df_train,df_val,df_test = data_selection(df)

In [6]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

train_loader = data_loading(df_train)
val_loader   = data_loading(df_val)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=None,
                    encoder_builder=builder,
                    n_classes=df.activity.nunique())

In [7]:
from torchsummary import summary

summary(model,(1,70,1600),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
             Stack-1          [-1, 3, 70, 1600]               0
UpsamplingNearest2d-2          [-1, 3, 70, 1600]               0
            Conv2d-3          [-1, 64, 16, 399]          23,296
              ReLU-4          [-1, 64, 16, 399]               0
         MaxPool2d-5           [-1, 64, 7, 199]               0
            Conv2d-6          [-1, 192, 7, 199]         307,392
              ReLU-7          [-1, 192, 7, 199]               0
         MaxPool2d-8           [-1, 192, 3, 99]               0
            Conv2d-9           [-1, 384, 3, 99]         663,936
             ReLU-10           [-1, 384, 3, 99]               0
           Conv2d-11           [-1, 256, 3, 99]         884,992
             ReLU-12           [-1, 256, 3, 99]               0
           Conv2d-13           [-1, 256, 3, 99]         590,080
             ReLU-14           [-1, 25

In [8]:
# train with poutyne
from validation.loov import leaveOneOut_crossValidation

records = leaveOneOut_crossValidation(model,df,transform,verbose=True)

One
Epoch: 1/1 Step: 26/26 100.00% |█████████████████████████|38.09s loss: 447.651918 acc: 31.506849 fscore_micro: 0.315068 fscore_macro: 0.151575 val_loss: 1.791747 val_acc: 16.666668 val_fscore_micro: 0.166667 val_fscore_macro: 0.047619
loss                447.651918
time                 38.094241
acc                  31.506849
fscore_micro          0.315068
fscore_macro          0.151575
val_loss              1.791747
val_acc              16.666668
val_fscore_micro      0.166667
val_fscore_macro      0.047619
Name: 0, dtype: float64 

Two
Epoch: 1/1 Step: 25/25 100.00% |█████████████████████████|37.71s loss: 276.575657 acc: 43.837357 fscore_micro: 0.438374 fscore_macro: 0.190038 val_loss: 1.792224 val_acc: 16.666668 val_fscore_micro: 0.166667 val_fscore_macro: 0.047619
loss                276.575657
time                 37.713494
acc                  43.837357
fscore_micro          0.438374
fscore_macro          0.190038
val_loss              1.792224
val_acc              16.666668


In [10]:
records

,loss,time,acc,fscore_micro,fscore_macro,val_loss,val_acc,val_fscore_micro,val_fscore_macro
0,447.651918,38.094241,31.506849,0.315068,0.151575,1.791747,16.666668,0.166667,0.047619
0,276.575657,37.713494,43.837357,0.438374,0.190038,1.792224,16.666668,0.166667,0.047619
0,687.670410,35.300305,32.591529,0.325915,0.081935,1.792861,16.666668,0.166667,0.047619
0,1374.277292,40.000762,38.488372,0.384884,0.206145,1.793488,16.666668,0.166667,0.047619
0,873.969257,38.414563,25.429116,0.254291,0.102881,1.792153,16.666668,0.166667,0.047619
0,479.837123,39.914294,27.329581,0.273296,0.103017,1.791377,16.666668,0.166667,0.047619


In [9]:
# average_score
records.mean()

loss                689.996943
time                 38.239610
acc                  33.197134
fscore_micro          0.331971
fscore_macro          0.139265
val_loss              1.792308
val_acc              16.666668
val_fscore_micro      0.166667
val_fscore_macro      0.047619
dtype: float64